# Football Scheduler

Here we implement the model descripted at [[1]](#References) and run some test with real teams.

## Imports

In [1]:
import json
import random
from typing import Callable, Tuple
from IPython.display import display, Markdown

from model import FootballSchedulerModel, SymetricScheme
from parse import parse_sol, to_df

In [2]:
countries = ["ARG", "BOL", "BRA", "CHI", "COL", "ECU", "PAR", "PER", "URU", "VEN"]
top_countries = ["ARG", "BRA"]
assert(set(top_countries).issubset(set(countries)))
n = len(countries)

In [3]:
def raffle(countries: list[str]) -> Tuple[Callable[[str], int], Callable[[int], str]]:
	countries_copy = countries.copy()
	random.shuffle(countries_copy)
	return lambda c: countries_copy.index(c), lambda i: countries_copy[i]

def write_raffle(base_path: str, index_of: Callable[[str], int], country_of: Callable[[int], str]):
	with open(f"{base_path}/index_of.json", "w") as f:
		json.dump(dict([(c, index_of(c)) for c in countries]), f)
	with open(f"{base_path}/country_of.json", "w") as f:
		json.dump(dict([(i, country_of(i)) for i in range(n)]), f)

# Represents the random asignment of country <-> id.
index_of, country_of = raffle(countries)

In [4]:
top_teams = [index_of(c) for c in top_countries]

## French Scheme

In [5]:
base_path_french = 'output/french'

In [6]:
write_raffle(base_path_french, index_of=index_of, country_of=country_of)
french_model = FootballSchedulerModel(n, SymetricScheme.FRENCH, top_teams)
french_model.write_problem(f"{base_path_french}/model.lp")

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/french/model.lp


In [7]:
sol_path = f"{base_path_french}/solution.sol"
french_model.optimize()
assert(french_model.get_obj_value() == 0)
french_model.write_sol(sol_path)

In [8]:
solution_path = f"{base_path_french}/solution.sol"
index_of_path = f"{base_path_french}/index_of.json"
country_of_path = f"{base_path_french}/country_of.json"

sol = parse_sol(f"{base_path_french}/solution.sol")
index_of = json.load(open(index_of_path))
country_of = json.load(open(country_of_path))

french_df = to_df(
    sol=sol,
    index_of=index_of,
    country_of=country_of,
)
display(Markdown(french_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| ARG    | BRA  | @CHI | @VEN | BOL  | URU  | @ECU | ARG  | @PER | @COL | CHI  | VEN  | @BOL | @URU | ECU  | @ARG | PER  | COL  | @BRA |
| BOL    | @URU | ECU  | @CHI | ARG  | BOL  | @PER | @BRA | VEN  | PAR  | @ECU | CHI  | @ARG | @BOL | PER  | BRA  | @VEN | @PAR | URU  |
| BRA    | @BOL | URU  | ECU  | @BRA | @ARG | COL  | @CHI | PAR  | VEN  | @URU | @ECU | BRA  | ARG  | @COL | CHI  | @PAR | @VEN | BOL  |
| CHI    | @VEN | PAR  | COL  | @URU | ECU  | @BRA | PER  | @ARG | BOL  | @PAR | @COL | URU  | @ECU | BRA  | @PER | ARG  | @BOL | VEN  |
| COL    | ARG  | @COL | @PER | VEN  | @CHI | PAR  | URU  | @BOL | @BRA | COL  | PER  | @VEN | CHI  | @PAR | @URU | BOL  | BRA  | @ARG |
| ECU    | @ECU | VEN  | BRA  | @COL | PER  | @BOL | @PAR | CHI  | URU  | @VEN | @BRA | COL  | @PER | BOL  | PAR  | @CHI | @URU | ECU  |
| PAR    | PER  | @BRA | URU  | @PAR | @COL | ARG  | @VEN | ECU  | @CHI | BRA  | @URU | PAR  | COL  | @ARG | VEN  | @ECU | CHI  | @PER |
| PER    | @PAR | BOL  | @ARG | PER  | @VEN | CHI  | COL  | @URU | ECU  | @BOL | ARG  | @PER | VEN  | @CHI | @COL | URU  | @ECU | PAR  |
| URU    | CHI  | @ARG | PAR  | @ECU | BRA  | @URU | BOL  | @COL | @PER | ARG  | @PAR | ECU  | @BRA | URU  | @BOL | COL  | PER  | @CHI |
| VEN    | COL  | @PER | @BOL | CHI  | @PAR | VEN  | @ECU | BRA  | @ARG | PER  | BOL  | @CHI | PAR  | @VEN | ECU  | @BRA | ARG  | @COL |

## References:
- [1] G. Durán, E. Mijangos, and M. Frisk, “Scheduling the South American qualifiers to the 2018 FIFA World Cup by integer programming,” European Journal of Operational Research, vol. 262, no. 3, pp. 1035–1048, 2017.
- [2] Pyscipopt Docs.